In [ ]:
# Downloading required libraries
!pip install -q transformers datasets sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 7.5 MB/s eta 0:00:00


In [ ]:
# Deleting the cache content of the transformer namespace
!rm -rf $HOME/.cache/huggingface/
!mkdir $HOME/.cache/huggingface/

In [ ]:
# Load the required libraries
# from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import os

In [ ]:
# Fine-tuning function
def fine_tune_t5(model_dir, num_iterations=5):
    # Load tokenizer and model
    tokenizer = T5Tokenizer.from_pretrained(model_dir)
    model = T5ForConditionalGeneration.from_pretrained(model_dir)

    # Set up optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = torch.nn.CrossEntropyLoss()

    # Fine-tune model for num_iterations
    for i in range(num_iterations):
        # Get context and question from user
        context = input("Enter context: ")
        question = input("Enter question: ")
        answer = input("Enter answer:")

        input_text = f'The answer for the question: "{question}" based on the context: "{context}"'
        output_text = f'{answer}'

        # Tokenize inputs
        input_ids = tokenizer.encode(input_text, return_tensors="pt", padding=True, truncation=True)
        target_ids = tokenizer.encode(output_text, return_tensors="pt", padding=True, truncation=True)

        # Forward pass
        outputs = model(input_ids=input_ids, labels=target_ids)
        loss = criterion(outputs.logits.view(-1, outputs.logits.shape[-1]), target_ids.view(-1))

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Print loss after each iteration
        print(f"Iteration {i+1}: Loss={loss.item()}")

        # Save model and tokenizer
        output_dir = os.path.join(os.path.dirname(model_dir), "fine_tuned_models", f"iteration_{i+1}")
        os.makedirs(output_dir, exist_ok=True)
        model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
        print(f"Model and tokenizer saved after iteration {i+1}")


In [ ]:
fine_tune_t5("/content/fine_tuned_model", 1)

Enter context: There are mysteries to the universe we were never meant to solve. But who we are and why we are here are not among. Those answers, we carry inside.
Enter question: Where do we carry answers?
Enter answer:Inside
Iteration 1: Loss=0.9629542827606201
Model and tokenizer saved after iteration 1


In [ ]:
# Testing Model - Local
# Set model to eval mode for inference
model.eval()
with torch.no_grad():
    # Get context from user
    context = input("Enter context: ")
    question = input("Enter question: ")
    input_text = f'The answer for the question: "{question}" based on the context: "{context}"'

    input_ids = tokenizer.encode(input_text, return_tensors="pt", padding=True, truncation=True)

    # Generate answer
    output_ids = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Print generated answer
    print(f"Generated answer: {question}")

Enter context: ou currently have zero compute units available. Resources offered free of charge are not guaranteed. Purchase more units here.
Enter question: How many compute units available?
Generated answer: zero compute units available
